In [ ]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
% cd /content/

/content


In [ ]:
!git clone https://github.com/tensorflow/tensor2tensor.git
#! pip install -q -U tensor2tensor==1.15.7

Cloning into 'tensor2tensor'...
remote: Enumerating objects: 33429, done.
remote: Counting objects: 100% (556/556), done.
remote: Compressing objects: 100% (502/502), done.
remote: Total 33429 (delta 139), reused 141 (delta 54), pack-reused 32873
Receiving objects: 100% (33429/33429), 17.20 MiB | 18.29 MiB/s, done.
Resolving deltas: 100% (26794/26794), done.


In [ ]:
%cd tensor2tensor
!pip install .

/content/tensor2tensor
Processing /content/tensor2tensor
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 5.8 MB 9.8 MB/s 
     |████████████████████████████████| 79 kB 8.6 MB/s 
     |████████████████████████████████| 193 kB 58.6 MB/s 
     |████████████████████████████████| 366 kB 44.3 MB/s 
     |████████████████████████████████| 48 kB 5.4 MB/s 
     |████████████████████████████████| 1.1 MB 46.2 MB/s 
     |████████████████████████████████| 367 kB 60.9 MB/s 
     |████████████████████████████████| 981 kB 37.9 MB/s 
     |████████████████████████████████| 352 kB 59.0 MB/s 
     |████████████████████████

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
#import tensorflow as tf
tf.__version__

'2.8.0'

In [ ]:
#import tensorflow as tf

# Enable Eager execution - useful for seeing the generated data.
tf.enable_eager_execution()

from tensor2tensor.utils import trainer_lib

# Set a seed so that we have deterministic outputs.
RANDOM_SEED = 301
trainer_lib.set_random_seed(RANDOM_SEED)

In [ ]:
#upload train data ligands file
from google.colab import files
files.upload()

In [ ]:
#upload train data proteins file
from google.colab import files
files.upload()

In [ ]:
!mv d2_tr_ligands.space_sep_seq ligands_train.space_sep_seq
!mv d2_tr_proteins.space_sep_seq proteins_train.space_sep_seq

In [ ]:
!ls

drive  sample_data  tensor2tensor


In [ ]:
import os

# Setup and create directories.
DATA_DIR = os.path.expanduser("/tmp/t2t/data")
OUTPUT_DIR = os.path.expanduser("/tmp/t2t/output")
TMP_DIR = os.path.expanduser("/tmp/t2t/tmp")
USR_DIR = os.path.expanduser("/tmp/t2t/usr")

# Create them.
tf.gfile.MakeDirs(DATA_DIR)
tf.gfile.MakeDirs(OUTPUT_DIR)
tf.gfile.MakeDirs(TMP_DIR)
tf.gfile.MakeDirs(USR_DIR)

In [ ]:
! mkdir transformer_scratch

In [ ]:
! cp /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/model/checkpoint /tmp/t2t/output
! cp /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/model/decode /tmp/t2t/output
! cp /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/model/e* /tmp/t2t/output
! cp /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/model/f* /tmp/t2t/output
! cp /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/model/g* /tmp/t2t/output
! cp /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/model/h* /tmp/t2t/output
! cp /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/model/model.ckpt-210937.* /tmp/t2t/output

cp: cannot stat '/content/drive/MyDrive/IITPKD System Data/home/iit/Research Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/model/decode': No such file or directory
cp: -r not specified; omitting directory '/content/drive/MyDrive/IITPKD System Data/home/iit/Research Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/model/eval'
cp: -r not specified; omitting directory '/content/drive/MyDrive/IITPKD System Data/home/iit/Research Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/model/eval_eval'


In [ ]:
% cd /tmp/t2t/data

/tmp/t2t/data


In [ ]:
#upload vocab file
from google.colab import files
files.upload()

Saving vocab.protein_specific_ligand_generation.tokens to vocab.protein_specific_ligand_generation.tokens


{'vocab.protein_specific_ligand_generation.tokens': b'<pad>_\n<EOS>_\nM\nA\nF\nK\nY\nL\nP\nI\nG\nS\nN\nE\nR\nQ\nV\nT\nH\nC\nD\nW\nc\n1\n(\n)\n=\nO\n2\n[\nn\n]\n-\n3\n4\n@\n5\n+\nl\nB\nr\n#\n\\\n/\ns\no\ne\na\nt\ng\nd\ni\n6\nb\n.\n7\n8\n9\n0\nu\nZ\n%\n'}

In [ ]:
% cd /content

/content


In [ ]:
%%writefile transformer_scratch/prob.py

import re

  
  
# `Problem` is the base class for any dataset that we want to add to T2T -- it
# unifies the specification of the problem for generating training data,
# training, evaluation and inference.
#
# All its methods (except `generate_data`) have reasonable default
# implementations.
#
# A sub-class must implement `generate_data(data_dir, tmp_dir)` -- this method
# is called by t2t-trainer or t2t-datagen to actually generate TFRecord dataset
# files on disk.
from tensor2tensor.data_generators import problem

# Certain categories of problems are very common, like where either the input or
# output is text, for such problems we define an (abstract) sub-class of
# `Problem` called `Text2TextProblem` -- this implements `generate_data` in
# terms of another function `generate_samples`. Sub-classes must override
# `generate_samples` and `is_generate_per_split`.
from tensor2tensor.data_generators import text_problems

# Every non-abstract problem sub-class (as well as models and hyperparameter
# sets) must be registered with T2T so that T2T knows about it and can look it
# up when you specify your problem on the commandline to t2t-trainer or
# t2t-datagen.
#
# One uses:
# `register_problem` for a new Problem sub-class.
# `register_model` for a new T2TModel sub-class.
# `register_hparams` for a new hyperparameter set. All hyperparameter sets
# typically extend `common_hparams.basic_params1` (directly or indirectly).
from tensor2tensor.utils import registry


# By default, when you register a problem (or model or hyperparameter set) the
# name with which it gets registered is the 'snake case' version -- so here
# the Problem class `ProteinSpecificLigandGeneration` will be registered with
# the name `protein_specific_ligand_generation`.
#
# One can override this default by actually assigning a name as follows:
# `@registry.register_problem("my_awesome_problem")`
#
# The registered name is specified to the t2t-trainer or t2t-datagen using the
# commandline flag `--problem`.
@registry.register_problem('protein_specific_ligand_generation')

# We inherit from `Text2TextProblem` which takes care of a lot of details
# regarding reading and writing the data to disk, what vocabulary type one
# should use, its size etc -- so that we need not worry about them, one can,
# of course, override those.


class ProteinSpecificLigandGeneration(text_problems.Text2TextProblem):
  

  # START: Methods we should override.

  # The methods that need to be overriden from `Text2TextProblem` are:
  # `is_generate_per_split` and
  # `generate_samples`.

  @property
  def is_generate_per_split(self):
    # If we have pre-existing data splits for (train, eval, test) then we set
    # this to True, which will have generate_samples be called for each of the
    # dataset_splits.
    #
    # If we do not have pre-existing data splits, we set this to False, which
    # will have generate_samples be called just once and the Problem will
    # automatically partition the data into dataset_splits.
    return False

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    # Here we are generating the data in-situ using the `sample_sentence`
    # function, otherwise we would have downloaded the data and put it in
    # `tmp_dir` -- and read it from that location.
    del tmp_dir

    # Unused here, is used in `Text2TextProblem.generate_data`.
    del data_dir

    # This would have been useful if `self.is_generate_per_split()` was True.
    # In that case we would have checked if we were generating a training,
    # evaluation or test sample. This is of type `problem.DatasetSplit`.
    del dataset_split

    
  #  drug_file = open ("drugs.smi.txt", "r")
  
  
   
    with open("/content/proteins_train.space_sep_seq") as file1, open("/content/ligands_train.space_sep_seq") as file2:
      for x, y in zip(file1, file2):
        inputs = x.strip()
        targets = y.strip()
        yield {"inputs":inputs,"targets":targets}
     
    
    
  # END: Methods we should override.

  # START: Overridable methods.

  @property
  def vocab_type(self):
    # We can use different types of vocabularies, `VocabType.CHARACTER`,
    # `VocabType.SUBWORD` and `VocabType.TOKEN`.
    #
    # SUBWORD and CHARACTER are fully invertible -- but SUBWORD provides a good
    # tradeoff between CHARACTER and TOKEN.
    return text_problems.VocabType.TOKEN
  



  @property
  def dataset_splits(self):
    # Since we are responsible for generating the dataset splits, we override
    # `Text2TextProblem.dataset_splits` to specify that we intend to keep
    # 80% data for training and 10% for evaluation and testing each.
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 144000,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 1,
    }, {
        "split": problem.DatasetSplit.TEST,
        "shards": 1,
    }]

 # END: Overridable methods.


Writing transformer_scratch/prob.py


In [ ]:
%%writefile transformer_scratch/__init__.py

from transformer_scratch import prob

Writing transformer_scratch/__init__.py


# Very Important!
## Do these steps to successfully run this code on TF2 version:

0.   Change train:val:test splits in above code cell.
1.   Go to content/tensor2tensor/tensor2tensor/bin/t2t-trainer
2.   Update file with below code
    ```
    from __future__ import absolute_import
    from __future__ import division
    from __future__ import print_function

    from tensor2tensor.bin import t2t_trainer

    import tensorflow.compat.v1 as tf

    def main(argv):
        t2t_trainer.main(argv)

    if __name__ == "__main__":
        tf.disable_v2_behavior()
        tf.logging.set_verbosity(tf.logging.INFO)
        tf.app.run(main)
    ```
3.  Go to content/tensor2tensor/tensor2tensor/bin/t2t-decoder
4.   Update file with below code
    ```
    """t2t-decoder."""
    from __future__ import absolute_import
    from __future__ import division
    from __future__ import print_function

    #import tensorflow.compat.v1 as tf
    from tensor2tensor.bin import t2t_decoder
    import logging
    #import tensorflow as tf
    import tensorflow.compat.v1 as tf

    def main(argv):
        t2t_decoder.main(argv)

    if __name__ == "__main__":
        tf.disable_v2_behavior()
        tf.logging.set_verbosity(tf.logging.INFO)
        tf.app.run()
    ```
5.   Go to usr/local/bin/t2t-trainer
6.   Update file with below code
    ```
    from __future__ import absolute_import
    from __future__ import division
    from __future__ import print_function

    from tensor2tensor.bin import t2t_trainer

    import tensorflow.compat.v1 as tf

    def main(argv):
        t2t_trainer.main(argv)

    if __name__ == "__main__":
        tf.disable_v2_behavior()
        tf.logging.set_verbosity(tf.logging.INFO)
        tf.app.run(main)
    ```
7.  Go to usr/local/bin/t2t-decoder
8.   Update file with below code
    ```
    """t2t-decoder."""
    from __future__ import absolute_import
    from __future__ import division
    from __future__ import print_function

    #import tensorflow.compat.v1 as tf
    from tensor2tensor.bin import t2t_decoder
    import logging
    #import tensorflow as tf
    import tensorflow.compat.v1 as tf

    def main(argv):
        t2t_decoder.main(argv)

    if __name__ == "__main__":
        tf.disable_v2_behavior()
        tf.logging.set_verbosity(tf.logging.INFO)
        tf.app.run()
    ```



In [ ]:
%%writefile /content/tensor2tensor/tensor2tensor/bin/t2t-trainer
#!/usr/bin/env python
"""Trainer for Tensor2Tensor.

This script is used to train your models in Tensor2Tensor.

For example, to train a shake-shake model on MNIST run this:

t2t-trainer \
  --generate_data \
  --problem=image_mnist \
  --data_dir=~/t2t_data \
  --tmp_dir=~/t2t_data/tmp
  --model=shake_shake \
  --hparams_set=shake_shake_quick \
  --output_dir=~/t2t_train/mnist1 \
  --train_steps=1000 \
  --eval_steps=100
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensor2tensor.bin import t2t_trainer

import tensorflow.compat.v1 as tf

def main(argv):
   t2t_trainer.main(argv)

if __name__ == "__main__":
   tf.disable_v2_behavior()
   tf.logging.set_verbosity(tf.logging.INFO)
   tf.app.run(main)

Overwriting /content/tensor2tensor/tensor2tensor/bin/t2t-trainer


In [ ]:
%%writefile /usr/local/bin/t2t-trainer
#!/usr/bin/env python
"""Trainer for Tensor2Tensor.

This script is used to train your models in Tensor2Tensor.

For example, to train a shake-shake model on MNIST run this:

t2t-trainer \
  --generate_data \
  --problem=image_mnist \
  --data_dir=~/t2t_data \
  --tmp_dir=~/t2t_data/tmp
  --model=shake_shake \
  --hparams_set=shake_shake_quick \
  --output_dir=~/t2t_train/mnist1 \
  --train_steps=1000 \
  --eval_steps=100
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensor2tensor.bin import t2t_trainer

import tensorflow.compat.v1 as tf

def main(argv):
   t2t_trainer.main(argv)

if __name__ == "__main__":
   tf.disable_v2_behavior()
   tf.logging.set_verbosity(tf.logging.INFO)
   tf.app.run(main)

Overwriting /usr/local/bin/t2t-trainer


In [ ]:
%%writefile /content/tensor2tensor/tensor2tensor/bin/t2t-decoder
#!/usr/bin/env python
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

#import tensorflow.compat.v1 as tf
from tensor2tensor.bin import t2t_decoder
import logging
#import tensorflow as tf
import tensorflow.compat.v1 as tf

def main(argv):
   t2t_decoder.main(argv)

if __name__ == "__main__":
   tf.disable_v2_behavior()
   tf.logging.set_verbosity(tf.logging.INFO)
   tf.app.run()

Overwriting /content/tensor2tensor/tensor2tensor/bin/t2t-decoder


In [ ]:
%%writefile /usr/local/bin/t2t-decoder
#!/usr/bin/env python
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

#import tensorflow.compat.v1 as tf
from tensor2tensor.bin import t2t_decoder
import logging
#import tensorflow as tf
import tensorflow.compat.v1 as tf

def main(argv):
   t2t_decoder.main(argv)

if __name__ == "__main__":
   tf.disable_v2_behavior()
   tf.logging.set_verbosity(tf.logging.INFO)
   tf.app.run()

Overwriting /usr/local/bin/t2t-decoder


In [ ]:
import time
startTime = int(round(time.time() / 60))

In [ ]:
! t2t-datagen \
  --problem=protein_specific_ligand_generation \
  --data_dir=/tmp/t2t/data \
  --tmp_dir=/tmp/t2t/tmp \
  --t2t_usr_dir=/content/transformer_scratch

INFO:tensorflow:Importing user module transformer_scratch from path /content
I0306 17:22:36.825704 140305123465088 usr_dir.py:43] Importing user module transformer_scratch from path /content
INFO:tensorflow:Generating problems:
    protein:
      * protein_specific_ligand_generation
I0306 17:22:36.829962 140305123465088 t2t_datagen.py:207] Generating problems:
    protein:
      * protein_specific_ligand_generation
INFO:tensorflow:Generating data for protein_specific_ligand_generation.
I0306 17:22:36.830313 140305123465088 t2t_datagen.py:280] Generating data for protein_specific_ligand_generation.
INFO:tensorflow:Generating case 0.
I0306 17:22:55.832134 140305123465088 generator_utils.py:178] Generating case 0.
INFO:tensorflow:Generating case 100000.
I0306 17:23:18.798233 140305123465088 generator_utils.py:178] Generating case 100000.
INFO:tensorflow:Generated 144000 Examples
I0306 17:26:42.345691 140305123465088 generator_utils.py:201] Generated 144000 Examples
INFO:tensorflow:Shuffli

In [ ]:
! t2t-trainer \
  --model=transformer \
  --hparams_set=transformer_tiny \
  --hparams="batch_size=4096,max_length=0,label_smoothing=0,learning_rate_decay_scheme='noam',num_hidden_layers=4" \
  --problem=protein_specific_ligand_generation \
  --worker_gpu=1 \
  --train_steps=210937 \
  --data_dir=/tmp/t2t/data \
  --output_dir=/tmp/t2t/output \
  --t2t_usr_dir=/content/transformer_scratch



Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Importing user module transformer_scratch from path /content
I0306 17:33:48.968021 140297283889024 usr_dir.py:43] Importing user module transformer_scratch from path /content
INFO:tensorflow:Loading hparams from existing json /tmp/t2t/output/hparams.json
I0306 17:33:48.969790 140297283889024 hparams_lib.py:64] Loading hparams from existing json /tmp/t2t/output/hparams.json
INFO:tensorflow:Overwrite key label_smoothing: 0.1 -> 0.0
I0306 17:33:48.973080 140297283889024 hparams_lib.py:86] Overwrite key label_smoothing: 0.1 -> 0.0
INFO:tensorflow:Overwrite key learning_rate_decay_scheme: noam -> 'noam'
I0306 17:33:48.973287 140297283889024 hparams_lib.py:86] Overwrite key learning_rate_decay_scheme: noam -> 'noam'
INFO:tensorflow:Overwrite key max_length: 256 -> 0
I0306 17:33:48.973473 140297283889024 hpa

In [ ]:
endTime = int(round(time.time() / 60))
print(f'Total time for training: {endTime - startTime} min')

Total time for training: 157 min


In [ ]:
%cd /tmp/t2t/output

/tmp/t2t/output


In [ ]:
!ls

checkpoint
eval
eval_eval
events.out.tfevents.1646063929.30abce71f66c
events.out.tfevents.1646074736.30abce71f66c
events.out.tfevents.1646259663.ef13386375a6
events.out.tfevents.1646504964.a917644a4ab1
events.out.tfevents.1646588070.b7b3250902ec
flags_t2t.txt
flags.txt
graph.pbtxt
hparams.json
model.ckpt-192000.data-00000-of-00002
model.ckpt-192000.data-00001-of-00002
model.ckpt-192000.index
model.ckpt-192000.meta
model.ckpt-193000.data-00000-of-00002
model.ckpt-193000.data-00001-of-00002
model.ckpt-193000.index
model.ckpt-193000.meta
model.ckpt-194000.data-00000-of-00002
model.ckpt-194000.data-00001-of-00002
model.ckpt-194000.index
model.ckpt-194000.meta
model.ckpt-195000.data-00000-of-00002
model.ckpt-195000.data-00001-of-00002
model.ckpt-195000.index
model.ckpt-195000.meta
model.ckpt-196000.data-00000-of-00002
model.ckpt-196000.data-00001-of-00002
model.ckpt-196000.index
model.ckpt-196000.meta
model.ckpt-197000.data-00000-of-00002
model.ckpt-197000.data-00001-of-00002
model.ckpt-197

In [ ]:
!cp checkpoint /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/
!cp decode /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/
!cp -r e* /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/
!cp f* /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/
!cp g* /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/
!cp h* /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/
!cp model.ckpt-210937.* /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/

cp: cannot stat 'decode': No such file or directory


In [ ]:
!ls

checkpoint
eval
eval_eval
events.out.tfevents.1646063929.30abce71f66c
events.out.tfevents.1646074736.30abce71f66c
events.out.tfevents.1646259663.ef13386375a6
events.out.tfevents.1646504964.a917644a4ab1
events.out.tfevents.1646588070.b7b3250902ec
flags_t2t.txt
flags.txt
graph.pbtxt
hparams.json
model.ckpt-192000.data-00000-of-00002
model.ckpt-192000.data-00001-of-00002
model.ckpt-192000.index
model.ckpt-192000.meta
model.ckpt-193000.data-00000-of-00002
model.ckpt-193000.data-00001-of-00002
model.ckpt-193000.index
model.ckpt-193000.meta
model.ckpt-194000.data-00000-of-00002
model.ckpt-194000.data-00001-of-00002
model.ckpt-194000.index
model.ckpt-194000.meta
model.ckpt-195000.data-00000-of-00002
model.ckpt-195000.data-00001-of-00002
model.ckpt-195000.index
model.ckpt-195000.meta
model.ckpt-196000.data-00000-of-00002
model.ckpt-196000.data-00001-of-00002
model.ckpt-196000.index
model.ckpt-196000.meta
model.ckpt-197000.data-00000-of-00002
model.ckpt-197000.data-00001-of-00002
model.ckpt-197

In [ ]:
%cd /content/

/content


In [ ]:
#upload test data proteins file
from google.colab import files
files.upload()

In [ ]:
!ls

d2_te_unique_proteins.txt  sample_data	  transformer_scratch
drive			   tensor2tensor


In [ ]:
!cp d2_te_unique_proteins.txt /tmp/t2t/data

In [ ]:
%cd /tmp/t2t/data

/tmp/t2t/data


In [ ]:
!ls

d2_te_unique_proteins.txt
proteins_test1.space_sep_seq_unique
vocab.protein_specific_ligand_generation.tokens


In [ ]:
import random

proteins = []
with open('d2_te_unique_proteins_run1.txt') as f:
    for line in f:
        if line.strip() not in proteins:
            proteins.append(line.strip())

with open('proteins_test1.space_sep_seq_unique', 'w+') as f:
    for p in proteins:
        sp = p
        print(sp)
        f.write(sp+'\n')

10
M L K F K Y G A R N P L D A G A A E P I A S R A S R L N L F F Q G K P P F M T Q Q Q M S P L S R E G I L D A L F V L F E E C S Q P A L M K I K H V S N F V R K Y S D T I A E L Q E L Q P S A K D F E V R S L V G C G H F A E V Q V V R E K A T G D I Y A M K V M K K K A L L A Q E Q V S F F E E E R N I L S R S T S P W I P Q L Q Y A F Q D K N H L Y L V M E Y Q P G G D L L S L L N R Y E D Q L D E N L I Q F Y L A E L I L A V H S V H L M G Y V H R D I K P E N I L V D R T G H I K L V D F G S A A K M N S N K M V N A K L P I G T P D Y M A P E V L T V M N G D G K G T Y G L D C D W W S V G V I A Y E M I Y G R S P F A E G T S A R T F N N I M N F Q R F L K F P D D P K V S S D F L D L I Q S L L C G Q K E R L K F E G L C C H P F F S K I D W N N I R N S P P P F V P T L K S D D D T S N F D E P E K N S W V S S S P C Q L S P S G F S G E E L P F V G F S Y S K A L G I L G R S E S V V S G L D S P A K T S S M E K K L L I K S K E L Q D S Q D K C H K M E Q E M T R L H R R V S E V E A V L S Q K E V E L K A S E T Q

In [ ]:
% cd /content

/content


In [ ]:
from tensor2tensor.bin import t2t_decoder

In [ ]:
import time
startTime = int(round(time.time() * 1000))

In [ ]:
! t2t-decoder \
  --hparams_set=transformer_tiny \
  --hparams="batch_size=4096,max_length=0,label_smoothing=0,learning_rate_decay_scheme='noam',num_hidden_layers=4" \
  --decode_hparams="beam_size=100,alpha=0.6,batch_size=4,write_beam_scores=False, return_beams=True" \
  --model=transformer \
  --problem=protein_specific_ligand_generation \
  --data_dir=/tmp/t2t/data \
  --output_dir=/tmp/t2t/output \
  --t2t_usr_dir=/content/transformer_scratch \
  --decode_from_file=/tmp/t2t/data/proteins_test1.space_sep_seq_unique \
  --decode_to_file=/content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/generated_samples/protein.seq.decode.results_ds2_ep6000_beam100

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
samples_dir = '/content/drive/MyDrive/IITPKD System Data/home/iit/Research Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/generated_samples/'
sample_size = 100

endTime = int(round(time.time()))
with open(samples_dir+f'samples_evaluation.txt', 'a+') as eval_f:
    eval_f.write(f'Total time for {sample_size} samples: {endTime - startTime} seconds\n')